---
title: "Using JavaScript and D3 in Jupyter"
author: "Vahram Poghosyan"
date: "2024-12-26"
categories: ["D3", "Visualization", "JavaScript"]
format:
  html:
    code-fold: false
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

By far the quickest, and dirtiest way to use D3 in Jupyter notebooks is using the Jupyter kernel. This poses disadvantages, namely that we need to write the JavaScript and D3 code non-interactively, then pass it as a raw string inside a `<script>` tag. We may stub some data fields (like `$data`) and, on-the-fly, switch out the stub with data gathered using Python. But we can also use [Deno](https://deno.com/), a JavaScript runtime environment from the maker of Node.js, as a kernel instead (more on that [here](./js_and_d3_in_jupyter_with_deno.ipynb))

Let's look at a minimal example.

In [22]:
from IPython.display import display, HTML


In [23]:
display(HTML('<p> Hello world! </p>'));

Let's get some simple styling options.

In [24]:
display(HTML('''
<style scoped>
.steely {
    color: steelblue;
    font: 16px script;
}
</style»
<p class="steely"> Hello world! </p>
'''
));

D3 is a JavaScript library for visualization. The D-s in D3 are for "Data Driven Documents." Think DOM elements that are styled, positioned, scaled, etc. based on data...

A core pattern in D3 is the following:

1. Select some array of DOM elements
2. 'Attach' an array of data to that array of DOM elements
3. Perform some actions for each DOM element formed from the data array

In [ ]:
// Imports: D3, TopoJSON, and Linkedom
import * as d3 from "npm:d3";
import * as topojson from "npm:topojson";
import { DOMParser } from "npm:linkedom";

// Create a fake DOM via Linkedom
const document = new DOMParser().parseFromString(
  `<!DOCTYPE html><html lang="en"><head></head><body></body></html>`,
  "text/html",
);

// Fetch and parse the US map TopoJSON
const url = "https://cdn.jsdelivr.net/npm/us-atlas@3/states-10m.json";
const us = await (await fetch(url)).json();
const states = topojson.feature(us, us.objects.states) as any; // GeoJSON 'FeatureCollection'

// Create an <svg> using Linkedom's fake DOM using D3
const width = 975;
const height = 610;
const body = d3.select(document.body);
body
  .append("p")
  .text("💡 Tip: Try hovering over a county!")
  .style("text-align", "center")
const svg = body
  .append("svg")
  .attr("xmlns", "http://www.w3.org/2000/svg") 
  .attr("width", width)
  .attr("height", height)
  .attr("viewBox", `0 0 ${width} ${height}`);
const mapGroup = svg.append("g");

// Create a geo-projection & path generator
const projection = d3.geoAlbersUsa().fitSize([width, height], states);
const path = d3.geoPath(projection);

const statesGroup = mapGroup.append("g")
  .attr("fill", "#EBE8E7")
  .attr("stroke", "#191516")

// Append a <path> per state
statesGroup.selectAll("path")
  .data(states.features)
  .join("path")
  // Add attributes to each <path>
  .on("mouseover", mouseover)
  .attr("cursor", "pointer")
  .attr("onmouseover", "this.style.fill = '#F9C22E';")
  .attr("onmouseout", "this.style.fill = '#EBE8E7';")
  .attr("d", path);

function mouseover(event, d) {
  console.log("HERE")
}